**Imports**

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator

from oracles_grover import marked_oracle

import numpy as np
import random
import math

from grover import Grover_Algorithm


**Parameter Tuning**

Manual Input

In [4]:
n=10
m_initial = 2
lambda_val=6/5
desired_success_rate=0.99

Calculated Params i.e dependent on manual params

In [6]:
N=n*(n-1)/2 
alpha=-math.log(1-desired_success_rate)/math.log(n) #Can use n or N
k=int(450*alpha*math.log(n)) 
num_qubits=math.ceil(math.log2(N))

In [7]:
print("N =", N)
print("Alpha =",alpha)
print("k =",k)
print("num_qubits =", num_qubits)

N = 45.0
Alpha = 1.9999999999999996
k = 2072
num_qubits = 6


**Marked States for Current Fake Oracle**

In [8]:

def random_bitstrings(num_bits: int, num_count: int):
    """Generate num_count unique random bitstrings of length num_bits."""
    if num_count > 2**num_bits:
        raise ValueError("Cannot generate more unique bitstrings than possible combinations.")

    seen = set()
    while len(seen) < num_count:
        bitstring = ''.join(random.choice('01') for _ in range(num_bits))
        seen.add(bitstring)
    return seen   



In [10]:
marked_states= random_bitstrings(num_qubits,10)
print(marked_states)

{'110100', '010000', '101111', '011011', '011101', '111011', '000010', '011001', '001001', '111101'}


**Algorithm Elements**

In [11]:
oracle=marked_oracle(marked_states, num_qubits=num_qubits)
backend= AerSimulator()

In [12]:
grover=Grover_Algorithm(oracle=oracle, backend=backend, optimization_level=0)

**First Stage**

In [13]:
m=m_initial
while True:
    n_success=0
    for i in range(k):
        j=random.randint(0,math.ceil(m)-1) #isn't it 0<j<m? and not 0<=j<m? what happens when m=1??
        qc=grover.create_grover_circuit(iterations=j)
        print(id(grover.oracle))
        result=grover.run(qc)
        if result in marked_states:
            n_success+=1
    if n_success/k>=0.2:
        break
    m=min(lambda_val*m,int(math.sqrt(N)))

5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176
5016644176

KeyboardInterrupt: 

Printing out updated m and number of successes in first precritical stage

In [77]:
print("n_success =",n_success)
print("Updated m =",m)
print("n_success/k =",n_success/k)

n_success = 531
Updated m = 5.9719679999999995
n_success/k = 0.2562741312741313


**Second Stage**

In [78]:
success=False
A=set()

while not success:
    n_success=0
    k=6*alpha*math.log(n)
    d=0
    while n_success<=k:
        j=random.randint(1,math.ceil(m)-1)
        qc=grover.create_grover_circuit(iterations=j)
        result=grover.run(qc)
        
        if result in marked_states:
            n_success+=1
            if result not in A:
                A.add(result)
                d+=1
    if d/k<0.5:
        success=True
    
u=4*len(A)

**Third Stage**

In [79]:
n_success=0
k=(alpha+1)*u*math.log(n)
while n_success<=k:
    j=random.randint(1,math.ceil(m)-1)
    qc=grover.create_grover_circuit(iterations=j)
    result=grover.run(qc)
    if result in marked_states:
            n_success+=1
            if result not in A:
                A.add(result)
print(A)

{'0011001111010', '0111011101000', '0111010101101', '1110101101000', '1001011000110', '0011110000110', '1111111101000', '1100110010111', '1111100001110', '0111000110101', '0001010010101', '1110001101100', '1000100010111', '1001011000010', '0100101101100', '0010100101010', '0000100011011', '1010011000010', '0000110100101', '1000100101100', '1001011001010', '0101010011001', '1011001011110', '1010100000001', '1010010111100', '0011101011100', '1011011010010', '0101000101111', '1101001001010', '0010011010000', '0001110001001', '1001001000101', '1111000010110', '0001001100000', '0111010111001', '0110101111001', '1011110010000', '1101001001000', '0010001101100', '0010101011001', '0101101101111', '0101100000010', '1000001110101', '0101100101001', '1100010110000', '0001010111101', '1110100011001', '1101010010011', '1011011010110', '1101111101111'}


**Success Determination**

In [80]:
matching = 0
for i in A:
    if i in marked_states:
        matching+=1
    
achieved_success_rate=matching/len(marked_states)
print(f"Achieved Success Rate: {achieved_success_rate*100}%")

Achieved Success Rate: 100.0%


In [81]:
print(len(marked_states))

50
